In [1]:
"""

L'algorithme R2_prev implemente une validation croise sur l'ensemble du jeu de donnees fournis

Arguments:
    Y : La variable d'interet
    W : Notre estimation de la variable d'interet
    X : Nos variables explicatives
    


Retourne:

    R2_prev : Une mesure bornee entre [-inf, 1] qui represente le pouvoir predictif de nos estimations

"""

def R2_prev(Y,W,X):
    
    # On multiplie X par soi meme
    X2 = np.dot(X.T,X)  
    
    # The variance-covariance
    C = np.linalg.inv(X2)
    
    # Hat Matrix (X*B = H*Y)
    H = np.dot(np.dot(X,C),X.T)
    
    # We are only interested in the (i,i) values
    H = np.diagonal(H)
    
    # Mean of Y
    y_S = np.sum(Y)/float(len(Y))
    
    num = 0.0
    for i in range(len(Y)):
        num += ((Y[i] - W[i])/float(1 - H[i]))**2

    denom = 0.0
    for i in range(len(Y)):
        denom += (Y[i] - y_S)**2
    
    R2_prev = (1 - num/denom)
    
    return R2_prev
    

In [2]:
"""

Selon le document MTH3302_CriteresProjet.pdf nous sommes evalues par notre root mean squared error (RMSE).
On trouve ici son implementation.

Arguments :
    Y : La variable d'interet
    W : Notre estimation de la variable d'interet
    
    
Retourne:
    rmse : L'erreur moyenne de nos predictions

"""

def RMSE(Y,W):

    n = len(Y)

    total = 0.0
    for i in range(n):
        total += (Y[i] - W[i])**2
        
    rmse = total/float(n)
        
    return rmse

In [3]:
"""
--- Variable Qualitative -> Table Binaire --- 

Afin de traiter nos variables qualitatives, nous avons une fonction qui la tranforme en table binaire 
avec un one bit encoder. 

    Disons que la variable explicative peut prendre 4 valeurs {Bleu, Vert, Jaune, Rouge},
        
          ID        X1
        ----------------
          1        Bleu
          2        Bleu
          3        Vert
          4        Jaune
          5        Vert
          6        Rouge          
         ...
    
    Suite a un encodage en 1 bit notre variable explicative X1 devient,
    
          ID        Bleu        Vert        Jaune        Rouge
        -------------------------------------------------------
          1          1           0            0            0
          2          1           0            0            0
          3          0           1            0            0
          4          0           0            1            0
          5          0           1            0            0
          6          0           0            0            1
                                ...
                                

Nous nous basons sur la fonction get_dummies() de la librairie Panda pour faire ceci.


Arguments,

    trainSet : le jeu de donnees de training
    testSet : le jeu de donnees de test
    varName : la variable qualitative que nous souhaitons transformer en tableau binaire
    
    
Retourne, 
    trainSet_new : le jeu de donnees de training ou la variable qualitative a ete remplacee par un tableau binaire
    testSet_new : le jeu de donnees de test ou la variable qualitative a ete remplacee par un tableau binaire

"""

def qualiToBinary(trainSet, testSet, varName):
    
    # Genere la table binaire de la variable qualitative
    trainVar_binary = pd.get_dummies(trainSet[varName],prefix=varName)
    testVar_binary = pd.get_dummies(testSet[varName],prefix=varName)
    
    # On convertit nos tableaux binaires en Dataframe
    trainVar_binary = pd.DataFrame(trainVar_binary)
    testVar_binary = pd.DataFrame(testVar_binary)
    
    # On enleve la variable qualitative de nos jeux de donnees
    trainSet_new = trainSet.drop([varName], axis=1)
    testSet_new = testSet.drop([varName], axis=1)   

    # On ajoute les nouvelles colonnes a nos jeux de donnees
    trainSet_new = pd.concat([trainSet_new,trainVar_binary],axis=1)
    testSet_new = pd.concat([testSet_new,testVar_binary],axis=1)
    
    # On identifie les colonnes manquantes dans le jeu de donnees test
    missing_categories = set(trainSet_new) - set(testSet_new)

    # On ajoute ces colonnes manquantes avec une valeur par defaut de 0
    for c in missing_categories:
        testSet_new[c] = 0

    # On identifie les colonnes manquantes dans le jeu de donnees training
    missing_categories = set(testSet_new) - set(trainSet_new)

    # On ajoute ces colonnes manquantes avec une valeur par defaut de 0
    for c in missing_categories:
        trainSet_new[c] = 0


    # On aligne nos jeux de donnees pour que les memes colonnes soient aux memes endroits
    trainSet_new, testSet_new = trainSet_new.align(testSet_new, axis=1)
    
    # On retourne les jeux de donnees
    return trainSet_new, testSet_new

In [4]:
def PCA(Y,X,varExplained):
        
    # On transforme X en une matrice de moyenne = 0 et variance = 1
    scaler = StandardScaler()
    X_std = scaler.fit_transform(X)
    nbrOfVar = X.shape[1]

    # On calcule la matrice de covariance
    E = np.dot(X_std.T,X_std)/(nbrOfVar-1)

    # On calcule nos Valeurs et Vecteurs propres
    eig_val, eig_vec = np.linalg.eig(E)
    eig_val = eig_val.real
    eig_vec = eig_vec.real
    eig_vec = eig_vec.T # pour les ordonner par range

    # On calcule la contribution de chaque valeur propre a la variance total
    eig_val_contribution = eig_val/np.sum(eig_val)

    # On cree une liste d'index de 0 au nombre de valeurs propres
    rangeIndex = np.array(range(len(eig_val_contribution)))

    # On cree un tableau indexe de nos valeurs propres et leur contribution a la variance
    eig_val_table = np.array(([rangeIndex,eig_val,eig_val_contribution])).T

    # On ordonne en ordre decroissant de contribution le tableau
    eig_val_table_ordered = eig_val_table[(-eig_val_table[:,2]).argsort()]
    
    # On rejoint la quantite de variance souhaitee
    varCumul = 0.0
    varIndex = 0
    for val in eig_val_table_ordered:
        varIndex += 1
        varCumul += val[2]
        if(varCumul >= varExplained):
            break
    
    print("Nbr of Var = %d" % varIndex)
    
    # On recupere les index des valeurs propres que nous garderont
    indexKept = eig_val_table_ordered[:,0][:varIndex]

    # Our reduced regressor
    K_vect = eig_vec[indexKept.astype(int),:].T

    # Our principal components
    Z = np.dot(X_std,K_vect)

    # Our Regression Coefficiants
    B_partial = np.dot(np.linalg.inv(np.dot(Z.T,Z)),Z.T)
    B = np.dot(B_partial,Y)

    # We calculate the intercept
    uY = np.mean(Y)

    # We add the intercept
    Y_new = np.dot(Z,B) + uY

    # Clip to min 0
    Y_new = Y_new.clip(min=0)
    
    return Y_new, B, K_vect, uY


In [5]:
def PCA_trained(X,B,K_vect,uY):
    
    X_std = StandardScaler().fit_transform(X)
    
    # Our principal components
    Z = np.dot(X_std,K_vect)
    
    # We add the intercept
    Y_new = np.dot(Z,B) + uY
    
    # Clip to min 0
    Y_new = Y_new.clip(min=0)

    return Y_new

In [6]:
# On importe les librairies
import numpy as np # calcul matriciel
import pandas as pd # structure des donnees
import matplotlib.pyplot as plt # graphiques
from sklearn.preprocessing import StandardScaler

# Chemin vers les jeux de donnees en format csv
pathTest = "./test3.csv"
pathTrain = "./train3.csv"

# On importe les jeux de donnees
df_test = pd.read_csv(pathTest, sep=',', index_col=0)
df_train = pd.read_csv(pathTrain, sep=',', index_col=0)

# On enleve la colonne Name qui ne sert pas a l'analyse
df_train = df_train.drop(['Name'],axis=1)

# --- Variables Explicatives ---

#    Name : Nom du jeu
#    Platform : Console sur laquelle le jeu fonctionne
#    Year_of_Release : Année de sortie du jeu
#    Genre
#    Publisher
#    JP_Sales : Nombre de ventes du jeu au Japon en millions d’unités
#    Other_Sales : Nombre de ventes du jeu ailleurs dans le monde : Afrique, Asie sans le Japon, Europe sans l’Union Européenne et Amérique du Sud en millions d’unités
#    Critic_Score : Score donné par Metacritic
#    Critic_Count : Nombre de critiques prises en compte pour estimer le Critic_score
#    User_Score : Score donné par les usagers de Metacritic
#    User_Count : Nombre d’usagers considérés pour estimer le User_Score
#    Developer : Compagnie créatrice du jeu
#    Rating : Classement ESRB (Entertainment Software Rating Board) ie à qui s’addresse le jeu (tout public, majeur, adolescents, etc) 


# --- Variables d'Interet ---

#    NA_sales : Nombre de ventes du jeu en Amérique du Nord en millions d’unités
#    Global_Sales : Nombre de ventes total du jeu en millions d’unités

In [7]:
#On donne une moyenne des Critic et User scores
df_train['Score'] = (df_train.Critic_Score + df_train.User_Score*10)/2
df_test['Score'] = (df_test.Critic_Score + df_test.User_Score*10)/2

#L'âge de la plateforme à la sortie du jeu
df_train['Platform_Age'] = df_train.Year_of_Release - df_train.groupby(['Platform'])['Year_of_Release'].transform(min)
df_test['Platform_Age'] = df_test.Year_of_Release - df_test.groupby(['Platform'])['Year_of_Release'].transform(min)

#La moyenne des ventes par plateforme et genre
df_train['Mean_Sales_by_Platform'] = df_train.groupby(['Platform'])['Other_Sales'].transform('mean')
df_train['Mean_Sales_by_Genre'] = df_train.groupby(['Genre'])['Other_Sales'].transform('mean')
df_test['Mean_Sales_by_Platform'] = df_test.groupby(['Platform'])['Other_Sales'].transform('mean')
df_test['Mean_Sales_by_Genre'] = df_test.groupby(['Genre'])['Other_Sales'].transform('mean')

#Le nombre de jeux sortis la même année
df_train['Num_Games_Same_Year'] = df_train.groupby(['Year_of_Release'])['Year_of_Release'].transform('count')
df_test['Num_Games_Same_Year'] = df_test.groupby(['Year_of_Release'])['Year_of_Release'].transform('count')


In [8]:
"""

Ici on transforme nos variables qualitatives en tableau binaire

"""

# Variable Qualitative #1 : Platform
print("Nbr. of categories Platform (train) : %d" %(df_train.Platform.nunique()))
print("Nbr. of categories Platform (test) : %d \n" %(df_test.Platform.nunique()))
df_train, df_test = qualiToBinary(df_train, df_test, 'Platform') # On binairise


# Variable Qualitative #2 : Genre
print("Nbr. of categories Genre (train) : %d" %(df_train.Genre.nunique()))
print("Nbr. of categories Genre (test) : %d \n" %(df_test.Genre.nunique()))
df_train, df_test = qualiToBinary(df_train, df_test,'Genre') # On binairise


# Variable Qualitative #3 : Publisher
print("Nbr. of categories Publisher (train) : %d" %(df_train.Publisher.nunique()))
print("Nbr. of categories Publisher (test) : %d \n" %(df_test.Publisher.nunique()))
df_train, df_test = qualiToBinary(df_train, df_test, 'Publisher') # On binairise
#df_train = df_train.drop(['Publisher'], axis=1)
#df_test = df_test.drop(['Publisher'], axis=1)

# Variable Qualitative #4 : Developer
print("Nbr. of categories Developer (train) : %d" %(df_train.Developer.nunique()))
print("Nbr. of categories Developer (test) : %d \n" %(df_test.Developer.nunique()))
#df_train, df_test = qualiToBinary(df_train, df_test,'Developer') # On binairise
df_train = df_train.drop(['Developer'], axis=1)
df_test = df_test.drop(['Developer'], axis=1)

# Variable Qualitative #5 : Rating
print("Nbr. of categories Rating (train) : %d" %(df_train.Rating.nunique()))
print("Nbr. of categories Rating (test) : %d \n" %(df_test.Rating.nunique()))
df_train, df_test = qualiToBinary(df_train, df_test,'Rating') # On binairise

Nbr. of categories Platform (train) : 30
Nbr. of categories Platform (test) : 27 

Nbr. of categories Genre (train) : 12
Nbr. of categories Genre (test) : 12 

Nbr. of categories Publisher (train) : 552
Nbr. of categories Publisher (test) : 280 

Nbr. of categories Developer (train) : 1593
Nbr. of categories Developer (test) : 677 

Nbr. of categories Rating (train) : 8
Nbr. of categories Rating (test) : 5 



In [9]:
"""

Ici on ajoute de nouvelles colonnes qui sont des combinaisons non-lineaires 

des autres colonnes ou de toute nouvelles informations.

"""

# On ajoute une colonne qui comptabilise le nombre de NaN pour chaque jeu video
nbrOfNaN_train = ~(df_train.T.iloc[:].isna().sum().astype(bool))
df_train['Contains_NaN'] = nbrOfNaN_train.astype(np.uint8)

nbrOfNaN_test = ~(df_test.T.iloc[:].isna().sum().astype(bool))
df_test['Contains_NaN'] = nbrOfNaN_test.astype(np.uint8)

df_train['JP_Sales2'] = df_train.JP_Sales**2
df_test['JP_Sales2'] = df_test.JP_Sales**2

df_train['Other_Sales2'] = df_train.Other_Sales**2
df_test['Other_Sales2'] = df_test.Other_Sales**2

df_train['JP_Sales_Other_Sales'] = df_train.Other_Sales*df_train.JP_Sales
df_test['JP_Sales_Other_Sales'] = df_test.Other_Sales*df_test.JP_Sales

df_train['Other_Sales_Year_of_Release'] = df_train.Other_Sales*df_train.Year_of_Release
df_test['Other_Sales_Year_of_Release'] = df_test.Other_Sales*df_test.Year_of_Release

df_train['JP_Sales_Year_of_Release'] = df_train.JP_Sales*df_train.Year_of_Release
df_test['JP_Sales_Year_of_Release'] = df_test.JP_Sales*df_test.Year_of_Release

df_train['JP_Sales_Year2_of_Release2'] = df_train.JP_Sales**2*df_train.Year_of_Release**2
df_test['JP_Sales_Year2_of_Release2'] = df_test.JP_Sales**2*df_test.Year_of_Release**2

df_train['Critic_Score9_Critic_Count'] = df_train.Critic_Score**9*df_train.Critic_Count
df_test['Critic_Score9_Critic_Count'] = df_test.Critic_Score**9*df_test.Critic_Count

df_train['Critic_Score12_Critic_Count'] = df_train.Critic_Score**12*df_train.Critic_Count
df_test['Critic_Score12_Critic_Count'] = df_test.Critic_Score**12*df_test.Critic_Count

df_train['Critic_Score_Critic_Count'] = df_train.Critic_Score*df_train.Critic_Count
df_test['Critic_Score_Critic_Count'] = df_test.Critic_Score*df_test.Critic_Count

df_train['User_Score_User_Count'] = df_train.User_Score*df_train.User_Count
df_test['User_Score_User_Count'] = df_test.User_Score*df_test.User_Count

df_train['User_Score2'] = df_train.User_Score**2
df_test['User_Score2'] = df_test.User_Score**2

df_train['User_Count2'] = df_train.User_Count**2
df_test['User_Count2'] = df_test.User_Count**2

df_train['Critic_Count2'] = df_train.Critic_Count**2
df_test['Critic_Count2'] = df_test.Critic_Count**2

df_train['Critic_Score2'] = df_train.Critic_Score**2
df_test['Critic_Score2'] = df_test.Critic_Score**2

In [19]:
df_train_clean = df_train.fillna(df_train.median())
df_test_clean = df_test.fillna(df_train.median())

In [20]:
# On fait une matrice de nos variables
X_train = df_train_clean.drop(['NA_Sales','Global_Sales'],axis=1)
X_test = df_test_clean.drop(['NA_Sales','Global_Sales'],axis=1)

# On isole nos variables d'interet
Y1 = df_train_clean.Global_Sales.values
Y2 = df_train_clean.NA_Sales.values

In [26]:
Y_new1, B1, K_vect1, uY1 = PCA(Y1,X_train,0.999999)
rmse1 = RMSE(Y1,Y_new1)

Y1_test = PCA_trained(X_test,B1, K_vect1, uY1)

Nbr of Var = 624


In [27]:
Y_new2, B2, K_vect2, uY2 = PCA(Y2,X_train,0.999999)
rmse2 = RMSE(Y2,Y_new2)

Y2_test = PCA_trained(X_test,B2, K_vect2, uY2)

Nbr of Var = 624


In [28]:
rmseTot = np.sqrt(rmse1**2 + rmse2**2)
print("RMSE = %.4f" % rmseTot)

RMSE = 0.4432


In [29]:
"""

Processing du jeu de donnees test

"""

# On vient prendre le dataframe comme canva
df_test_estimated = pd.DataFrame([df_test.NA_Sales,df_test.Global_Sales]).copy().T

# On assigne nos predictions aux colonnes
df_test_estimated['Global_Sales'] = Y1_test
df_test_estimated['NA_Sales'] = Y2_test

# On sauvegarde nos predictions
df_test_estimated.to_csv("test_final.csv")